In [1]:
import cv2
import numpy as np
import scipy
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import glob
import matlab.engine
from tqdm import tqdm

In [ ]:
folders = glob.glob('*to*')

for folder in folders:
    os.chdir(folder)
    
    eng = matlab.engine.start_matlab()
    
    mylist = glob.glob('*.png')
    frames = []
    
    for fname in tqdm(mylist[:]):
        eng.workspace['s'] = fname
        inp = eng.eval("double(s)")
        out = eng.Wadell_python_v2(inp)
        data = {fname[:-4]:list(out[0])}
        frame = pd.DataFrame(data)
        frames.append(frame)
    
    results = pd.concat(frames, axis=1)

    results = results.transpose()
    results.columns = ['Roundness', 'sphericity1', 'sphericity2', 'sphericity3', 'sphericity4', 'sphericity5', 'rcum', 'size(boundary_points,1)', 'size(boundary_points)','2']
    results.index = mylist
    
    os.chdir("..")
    
    file_name = folder + '.csv'
    results.to_csv(file_name, index=True)

eng.quit()

In [9]:
files = glob.glob('????to???.csv')
files = files[10:]
original = pd.read_csv('Original_SRC-J.csv')

tmp = pd.read_csv(files[0])

df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()

df1 = tmp[['Unnamed: 0']]
df2 = tmp[['Unnamed: 0']]
df3 = tmp[['Unnamed: 0']]
df4 = tmp[['Unnamed: 0']]
df5 = tmp[['Unnamed: 0']]
df6 = tmp[['Unnamed: 0']]

for file in files:
    temp = pd.read_csv(file)
    name = file[0:-4]
    
    temp1 = temp['sphericity1']
    temp2 = temp['sphericity2']
    temp3 = temp['sphericity3']
    temp4 = temp['sphericity4']
    temp5 = temp['sphericity5']
    temp6 = temp['Roundness']
    
    temp1.name = name
    temp2.name = name
    temp3.name = name
    temp4.name = name
    temp5.name = name
    temp6.name = name
    
    df1 = pd.concat([df1, temp1], axis = 1)
    df2 = pd.concat([df2, temp2], axis = 1)
    df3 = pd.concat([df3, temp3], axis = 1)
    df4 = pd.concat([df4, temp4], axis = 1)
    df5 = pd.concat([df5, temp5], axis = 1)
    df6 = pd.concat([df6, temp6], axis = 1)

ori1 = original['sphericity1']
ori2 = original['sphericity2']
ori3 = original['sphericity3']
ori4 = original['sphericity4']
ori5 = original['sphericity5']
oriR = original['Roundness']

ori1.name = 'Original'
ori2.name = 'Original'
ori3.name = 'Original'
ori4.name = 'Original'
ori5.name = 'Original'
oriR.name = 'Original'

df1 = pd.concat([df1, ori1], axis = 1)
df2 = pd.concat([df2, ori2], axis = 1)
df3 = pd.concat([df3, ori3], axis = 1)
df4 = pd.concat([df4, ori4], axis = 1)
df5 = pd.concat([df5, ori5], axis = 1)
df6 = pd.concat([df6, oriR], axis = 1)    

df1.to_csv('Integrated-Area_Sphericity2.csv',index=False)
df2.to_csv('Integrated-Diameter_Sphericity2.csv',index=False)
df3.to_csv('Integrated-Circle_Ratio_Sphericity2.csv',index=False)
df4.to_csv('Integrated-Perimeter_Sphericity2.csv',index=False)
df5.to_csv('Integrated-Width_to_Length_Ratio_Sphericity2.csv',index=False)
df6.to_csv('Integrated-Roundness2.csv',index=False)

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [4]:
def Figure_plot(file_path):
    
    name = file_path[11:-4] + ' R-Square_RTO'
    os.makedirs(name, exist_ok=True)
    
    box1 = {'boxstyle': 'square',
            'ec': (0, 0, 0),
            'fc': (1, 1, 1),
            'linestyle': '-'}
    font1 = {'color':  'black',
             'weight': 'bold',
             'size': 16}
    
    data = pd.read_csv(file_path)
    r_square_df = pd.DataFrame(columns=['Phase', 'R-square'])
    
    column_names = [col for col in data.columns if 'Original' not in col and col != 'Unnamed: 0']

    # Plotting all the figures and calculating R-square values
    for col_name in column_names:
        plt.figure(figsize=(6,6))
    
        # Plotting
        plt.scatter(data['Original'], data[col_name], label=col_name)
        
        
        # Fitting Linear Regression Without Intercept
        X = data['Original'].values.reshape(-1, 1)  # Reshaping for sklearn
        y = data[col_name].values
        reg = LinearRegression(fit_intercept=False).fit(X, y)
        slope = reg.coef_[0]

        # Predicting
        y_pred = reg.predict(X)

        # Plotting the regression line
        plt.plot(data['Original'], y_pred, color='red', linewidth=2, label=f'Trend Line (y={round(slope, 3)}x)')

        # R-square
        r_square = r2_score(y, y_pred)
    
        # Creating a new DataFrame to concatenate
        new_row_df = pd.DataFrame({'Phase': [col_name], 'R-square': [r_square]})
        r_square_df = pd.concat([r_square_df, new_row_df], ignore_index=True)
    
        # Plot details
        plt.title(f'Linear Regression for {col_name}')
        plt.xlabel('Original')
        plt.ylabel(col_name)
        plt.xlim(0,1)
        plt.ylim(0, 1)
        
        plt.legend()
        
        text = f'R-Square: {round(r_square, 3)}'
        plt.text(0.5, 0.1, text, fontdict=font1, bbox=box1)
        
        save_path = os.path.join(name, f'Linear Regression for {col_name}_RTO')
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0, facecolor='white', dpi=350)
        
        plt.close()
    
    name = file_path[11:-4] + ' R-Square_RTO.csv'
    r_square_df.to_csv(name, index=False)

In [6]:
csvs = glob.glob('Integrated*.csv')

for csv in tqdm(csvs):
    Figure_plot(csv)

100%|██████████| 6/6 [01:16<00:00, 12.76s/it]
